# First Term Project: Cranfield Collection
“The Cranfield collection [...] was the pioneering test collection in allowing CRANFIELD precise quantitative measures of information retrieval effectiveness [...]. Collected in the United Kingdom starting in the late 1950s, it contains 1398 abstracts of aerodynamics journal articles, a set of 225 queries, and exhaustive relevance judgments of all (query, document) pairs.” [1, Section 8.2]

Your tasks, reviewed by your colleagues and the course instructors, are the following:

1.   *Implement an unsupervised ranked retrieval system*, [1, Chapter 6] which will produce a list of documents from the Cranfield collection in a descending order of relevance to a query from the Cranfield collection. You MUST NOT use relevance judgements from the Cranfield collection in your information retrieval system. Relevance judgements MUST only be used for the evaluation of your information retrieval system.

2.   *Document your code* in accordance with [PEP 257](https://www.python.org/dev/peps/pep-0257/), ideally using [the NumPy style guide](https://numpydoc.readthedocs.io/en/latest/format.html#docstring-standard) as seen in the code from exercises.  
     *Stick to a consistent coding style* in accordance with [PEP 8](https://www.python.org/dev/peps/pep-0008/).

3.   *Reach at least 22% mean average precision* [1, Section 8.4] with your system on the Cranfield collection. You MUST record your score either in [the public leaderboard](https://docs.google.com/spreadsheets/d/e/2PACX-1vT0FoFzCptIYKDsbcv8LebhZDe_20GFeBAPmS-VyImlWbqET0T7I2iWy59p9SHbUe3LX1yJMhALPcCY/pubhtml) or in this Jupyter notebook. You are encouraged to use techniques for tokenization, [1, Section 2.2] document representation [1, Section 6.4], tolerant retrieval [1, Chapter 3], relevance feedback and query expansion, [1, Chapter 9] and others discussed in the course.

4.   _[Upload an .ipynb file](https://is.muni.cz/help/komunikace/spravcesouboru#k_ss_1) with this Jupyter notebook to the homework vault in IS MU._ You MAY also include a brief description of your information retrieval system and a link to an external service such as [Google Colaboratory](https://colab.research.google.com/), [DeepNote](https://deepnote.com/), or [JupyterHub](https://iirhub.cloud.e-infra.cz/).

[1] Manning, Christopher D., Prabhakar Raghavan, and Hinrich Schütze. [*Introduction to information retrieval*](https://nlp.stanford.edu/IR-book/pdf/irbookonlinereading.pdf). Cambridge university press, 2008.

## Loading the Cranfield collection

First, we will install [our library](https://gitlab.fi.muni.cz/xstefan3/pv211-utils) and load the Cranfield collection.

In [ ]:
%%capture
! pip install git+https://github.com/MIR-MU/pv211-utils.git@spring2022
! pip install gensim==3.6.0

### Loading the documents

Next, we will define a class named `Document` that will represent a preprocessed document from the Cranfield collection. Tokenization and preprocessing of the `title` and `body` attributes of the individual documents as well as the creative use of the `authors`, `bibliography`, and `title` attributes is left to your imagination and craftsmanship.

In [ ]:
from typing import List

from pv211_utils.cranfield.entities import CranfieldDocumentBase

from gensim.utils import simple_preprocess

class Document(CranfieldDocumentBase):
    """
    A preprocessed Cranfield collection document.

    Parameters
    ----------
    document_id : str
        A unique identifier of the document.
    authors : list of str
        A unique identifiers of the authors of the document.
    bibliography : str
        The bibliographical entry for the document.
    title : str
        The title of the document.
    body : str
        The abstract of the document.

    """
    def __init__(self, document_id: str, authors: str, bibliography: str, title: str, body: str):
        super().__init__(document_id, authors, bibliography, title, body)

We will load documents into the `documents` [ordered dictionary](https://docs.python.org/3.8/library/collections.html#collections.OrderedDict). Each document is an instance of the `Document` class that we have just defined.

In [ ]:
from pv211_utils.cranfield.loader import load_documents

documents = load_documents(Document)

In [ ]:
print('\n'.join(repr(document) for document in list(documents.values())[:3]))
print('...')
print('\n'.join(repr(document) for document in list(documents.values())[-3:]))

In [ ]:
document = documents['14']
document

In [ ]:
print(document.authors)

In [ ]:
print(document.bibliography)

In [ ]:
print(document.title)

In [ ]:
print(document.body)

### Loading the queries
Next, we will define a class named `Query` that will represent a preprocessed query from the Cranfield collection. Tokenization and preprocessing of the `body` attribute of the individual queries is left to your craftsmanship.

In [ ]:
from pv211_utils.cranfield.entities import CranfieldQueryBase

class Query(CranfieldQueryBase):
    """
    A preprocessed Cranfield collection query.

    Parameters
    ----------
    query_id : int
        A unique identifier of the query.
    body : str
        The text of the query.

    """
    def __init__(self, query_id: int, body: str):
        super().__init__(query_id, body)

We will load queries into the `queries` [ordered dictionary](https://docs.python.org/3.8/library/collections.html#collections.OrderedDict). Each query is an instance of the `Query` class that we have just defined.

In [ ]:
from pv211_utils.cranfield.loader import load_queries

queries = load_queries(Query)

In [ ]:
print('\n'.join(repr(query) for query in list(queries.values())[:3]))
print('...')
print('\n'.join(repr(query) for query in list(queries.values())[-3:]))

In [ ]:
query = queries[14]
query

In [ ]:
print(query.body)

## Implementation of your information retrieval system
Next, we will define a class named `IRSystem` that will represent your information retrieval system. Your class must define a method name `search` that takes a query and returns documents in descending order of relevance to the query.

The example implementation returns documents in decreasing order of the bag-of-words cosine similarity between the document and the query. The example implementation returns documents in decreasing order of the TF-IDF cosine similarity between the document and the query. You can use the example implementation as a basis of your system, or you can replace it with your own implementation.

In [ ]:
from typing import Iterable

from pv211_utils.cranfield.irsystem import CranfieldIRSystemBase

from gensim.corpora import Dictionary
from gensim.matutils import cossim
from gensim.similarities import SparseMatrixSimilarity
from gensim.utils import simple_preprocess
from tqdm import tqdm

class IRSystem(CranfieldIRSystemBase):
    """
    A system that returns documents ordered by decreasing cosine similarity.

    Attributes
    ----------
    dictionary: Dictionary
        The dictionary of the system.
    index: MatrixSimilarity
        The indexed documents.
    index_to_document: dict of (int, Document)
        A mapping from indexed document numbers to documents.

    """
    def __init__(self):
        document_bodies = (simple_preprocess(document.body) for document in documents.values())
        document_bodies = tqdm(document_bodies, desc='Building the dictionary', total=len(documents))
        dictionary = Dictionary(document_bodies)
        document_vectors = (dictionary.doc2bow(simple_preprocess(document.body)) for document in documents.values())
        document_vectors = tqdm(document_vectors, desc='Building the index', total=len(documents))
        index = SparseMatrixSimilarity(document_vectors, num_docs=len(documents), num_terms=len(dictionary))
        index_to_document = dict(enumerate(documents.values()))
        self.dictionary = dictionary
        self.index = index
        self.index_to_document = index_to_document

    def search(self, query: Query) -> Iterable[Document]:
        """The ranked retrieval results for a query.

        Parameters
        ----------
        query : Query
            A query.
        
        Returns
        -------
        iterable of Document
            The ranked retrieval results for a query.

        """
        results = []
        query_vector = self.dictionary.doc2bow(simple_preprocess(query.body))
        similarities = enumerate(self.index[query_vector])
        similarities = sorted(similarities, key=lambda item: item[1], reverse=True)
        for document_number, _ in similarities:
            document = self.index_to_document[document_number]
            yield document

## Evaluation
Finally, we will evaluate your information retrieval system using [the Mean Average Precision](https://en.wikipedia.org/wiki/Evaluation_measures_(information_retrieval)#Mean_average_precision) (MAP) evaluation measure.

In [ ]:
from pv211_utils.cranfield.loader import load_judgements
from pv211_utils.cranfield.leaderboard import CranfieldLeaderboard
from pv211_utils.cranfield.eval import CranfieldEvaluation

submit_result = False
author_name = 'Surname, Name'

print('Initializing your system ...')
system = IRSystem()

evaluation = CranfieldEvaluation(system, load_judgements(queries, documents), CranfieldLeaderboard(), author_name)
evaluation.evaluate(tqdm(queries.values(), desc='Querying your system'), submit_result)